In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import optuna

/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [3]:
# Define the CNN model
class MyCNN(nn.Module):
    def __init__(self, dropout_rate):
        super(MyCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # Assume RGB images
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(32 * 16 * 16, 512),  # Flatten size depends on input image size
            nn.ReLU(),
            nn.Linear(512, 10)  # Adjust based on the number of classes in your dataset
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
# Objective function for Optuna to optimize
def objective(trial):
    # Hyperparameters to tune
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    lr = trial.suggest_loguniform('lr', 1e-4, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64])

    # Transformations and data loading
    transform = transforms.Compose([
        transforms.Resize((64, 64)),  # Resize images to 64x64
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    train_dataset = datasets.ImageFolder(train_dir, transform=transform)
    val_dataset = datasets.ImageFolder(val_dir, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MyCNN(dropout_rate).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Training loop
    for epoch in range(10):  # Reduced epochs for demonstration
        model.train()
        for data, targets in train_loader:
            data, targets = data.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    # Evaluation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(device), targets.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item(  accuracy = correct / total
    return accuracy

In [5]:
# Creating the Optuna study and running optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)  # You can increase or decrease the number of trials

print('Best hyperparameters:', study.best_trial.params)

[I 2024-11-01 00:31:30,710] A new study created in memory with name: no-name-a822bf4a-4f37-42da-a3ef-d536383af0e0
/var/folders/_0/n_t3__qd6k15bv63l5bktb8m0000gn/T/ipykernel_86418/288307961.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-1)
[I 2024-11-01 00:32:18,345] Trial 0 finished with value: 0.5714285714285714 and parameters: {'dropout_rate': 0.4070816891051241, 'lr': 0.021073229223398205, 'batch_size': 32}. Best is trial 0 with value: 0.5714285714285714.
[I 2024-11-01 00:33:07,444] Trial 1 finished with value: 0.5357142857142857 and parameters: {'dropout_rate': 0.26693458078077825, 'lr': 0.0006268491965006621, 'batch_size': 16}. Best is trial 0 with value: 0.5714285714285714.
[I 2024-11-01 00:33:57,302] Trial 2 finished with value: 0.5 and parameters: {'dropout_rate': 0.2031

Best hyperparameters: {'dropout_rate': 0.4070816891051241, 'lr': 0.021073229223398205, 'batch_size': 32}
